In [ ]:
## Model

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Streamlit UI
st.title("Stock Price Prediction using LSTM")
st.write("This app predicts the next day's closing price for the given stock data.")

# Upload CSV
uploaded_file = st.file_uploader("Upload a CSV file of stock data", type="csv")

if uploaded_file is not None:
    # Load data
    df = pd.read_csv(uploaded_file)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Prepare data for LSTM
    data = df['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Define training data
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    # Create sequences for LSTM
    def create_sequences(data, sequence_length):
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data[i:i + sequence_length])
            y.append(data[i + sequence_length])
        return np.array(X), np.array(y)

    sequence_length = 60
    X_train, y_train = create_sequences(train_data, sequence_length)
    X_test, y_test = create_sequences(test_data, sequence_length)

    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=0)

    # Predictions
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    # Combine predictions and actual values
    all_predictions = np.concatenate([train_predictions, test_predictions])
    all_actual = np.concatenate([y_train, y_test])

    # Inverse transform combined predictions and actual values
    all_predictions = scaler.inverse_transform(all_predictions)
    all_actual = scaler.inverse_transform(all_actual.reshape(-1, 1))

    # RMSE Calculation
    rmse = np.sqrt(mean_squared_error(all_actual, all_predictions))
    rae = np.sum(np.abs(all_actual - all_predictions)) / np.sum(np.abs(all_actual - np.mean(all_actual)))

    st.write(f"RMSE: {rmse}")
    st.write(f"RAE: {rae}")

    # Plot Actual vs Predicted
    st.write("### Actual vs Predicted Prices")
    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(df.index[-len(all_actual):], all_actual, label='Actual', color='blue')
    ax.plot(df.index[-len(all_actual):], all_predictions, label='Predicted', color='red')
    ax.set_title('Actual vs Predicted Prices')
    ax.set_xlabel('Date')
    ax.set_ylabel('Stock Price')
    ax.legend()
    st.pyplot(fig)

    # Predict next day closing price
    last_sequence = scaled_data[-sequence_length:]
    next_day_prediction = model.predict(last_sequence.reshape(1, sequence_length, 1))
    next_day_prediction = scaler.inverse_transform(next_day_prediction)

    st.write("### Predicted Next Day Closing Price")
    st.write(f"Next Day Closing Price: {next_day_prediction[0][0]:.2f}")
else:
    st.write("Please upload a CSV file to proceed.")

2024-12-19 18:24:17.670 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
